<a href="https://colab.research.google.com/github/dudumlc/ML_Regression/blob/main/ML_Previs%C3%A3o_Pre%C3%A7o_Carros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTAÇÃO DAS BIBLIOTECAS A SEREM USADAS

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, scale
from ipywidgets import widgets, interact
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

## IMPORTAÇÃO DO DATASET E PRÉ-VISUALIZAÇÃO

In [ ]:
df_raw = pd.read_csv('/content/C+¦pia de CarPrice_Assignment.csv')

In [ ]:
# Copiando outro df para trabalhar para ter o dataset bruto como backup se necessário
df = df_raw.copy()

In [ ]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


## DATA CLEANING

In [ ]:
# Conferência se há dados nulos inclusos nas colunas
df.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [ ]:
# Conferência se há linhas duplicadas inclusas
df.duplicated().sum()

0

In [ ]:
# Análise dos tipos de dados de cada coluna
df.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [ ]:
# Alterando tipo 'object' para 'category' (mais adequado e ainda economiza memória)
object_cols = df.select_dtypes(include='object').columns
df[object_cols] = df[object_cols].astype('category')

In [ ]:
# Colunas com tipos de dados alterados
df.dtypes

car_ID                 int64
symboling              int64
CarName             category
fueltype            category
aspiration          category
doornumber          category
carbody             category
drivewheel          category
enginelocation      category
wheelbase            float64
carlength            float64
carwidth             float64
carheight            float64
curbweight             int64
enginetype          category
cylindernumber      category
enginesize             int64
fuelsystem          category
boreratio            float64
stroke               float64
compressionratio     float64
horsepower             int64
peakrpm                int64
citympg                int64
highwaympg             int64
price                float64
dtype: object

In [ ]:
# Análise de outliers nas colunas numéricas - TABULARMENTE
def is_outlier(array, extreme=False):
  q1,q3 = np.quantile(array, [.25,.75])
  iqr = q3-q1

  factor = 3. if extreme else 1.5
  upper_outlier = q3 + factor*iqr
  lower_outlier = q1 - factor*iqr

  return (array < lower_outlier) | (array > upper_outlier)

num_cols = df.select_dtypes(include='number').columns
outliers = df[num_cols].apply(is_outlier).sum().to_frame('outliers').reset_index()
outliers = outliers[outliers['outliers'] > 0]
outliers

,index,outliers
2,wheelbase,3
3,carlength,1
4,carwidth,8
7,enginesize,10
9,stroke,20
10,compressionratio,28
11,horsepower,6
12,peakrpm,2
13,citympg,2
14,highwaympg,3


In [ ]:
@interact(method=outliers['index'])
def outlier(method):
  sns.histplot(data=df,x=method,kde=True)
  plt.show()

interactive(children=(Dropdown(description='method', options=('wheelbase', 'carlength', 'carwidth', 'enginesiz…

In [ ]:
# Retirando outliers para não impactar
df = df[~df[['compressionratio','horsepower']].apply(is_outlier).any(axis=1)].copy()

## DATA WRANGLING

In [ ]:
# Análise da granularidade de cada coluna categórica para definir a estratégia de transformação dessas categorias

for i in df.select_dtypes(include='category'):
  print("Coluna:",i)
  print("Categorias e quantidades:",df[i].unique())
  print()

Coluna: CarName
Categorias e quantidades: ['alfa-romero giulia', 'alfa-romero stelvio', 'alfa-romero Quadrifoglio', 'audi 100 ls', 'audi 100ls', ..., 'volvo 144ea', 'volvo 244dl', 'volvo 245', 'volvo 264gl', 'volvo diesel']
Length: 125
Categories (147, object): ['Nissan versa', 'alfa-romero Quadrifoglio', 'alfa-romero giulia',
                           'alfa-romero stelvio', ..., 'volvo 264gl', 'volvo diesel', 'vw dasher',
                           'vw rabbit']

Coluna: fueltype
Categorias e quantidades: ['gas']
Categories (2, object): ['diesel', 'gas']

Coluna: aspiration
Categorias e quantidades: ['std', 'turbo']
Categories (2, object): ['std', 'turbo']

Coluna: doornumber
Categorias e quantidades: ['two', 'four']
Categories (2, object): ['four', 'two']

Coluna: carbody
Categorias e quantidades: ['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop']
Categories (5, object): ['convertible', 'hardtop', 'hatchback', 'sedan', 'wagon']

Coluna: drivewheel
Categorias e quantidades: ['r

In [ ]:
# Coluna doornumber e cylindernumber tratam-se de número
df[['doornumber','cylindernumber']] = df[['doornumber','cylindernumber']].replace({'one':1,
                                                                                   'two':2,
                                                                                   'three':3,
                                                                                   'four':4,
                                                                                   'five':5,
                                                                                   'six':6,
                                                                                   'seven':7,
                                                                                   'eight':8,
                                                                                   'nine':9,
                                                                                   'ten':10})

In [ ]:
# Análise das escalas de cada coluna para ver a necessidade de normalização dos dados
df.describe().loc[['min','max']]

,car_ID,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
min,1.0,-2.0,86.6,141.1,60.3,48.8,1488.0,61.0,2.54,2.07,7.5,48.0,4200.0,14.0,16.0,5118.0
max,205.0,3.0,120.9,208.1,72.0,59.8,4066.0,308.0,3.94,4.17,10.1,184.0,6600.0,49.0,54.0,45400.0


In [ ]:
# As escalas são muito diferentes e precisará de normalização. Contudo, é necessário splitar o df em treino e teste primeiro para não haver data leakage

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
# Colunas categóricas restantes serão transformadas em dummies
X_train = pd.get_dummies(X_train, columns=df.select_dtypes(include='category').columns)
X_test = pd.get_dummies(X_test, columns=df.select_dtypes(include='category').columns)

In [ ]:
# Normalização dos dados para a diferença de escala entre as colunas não atrapalhe a regressão (NORMALIZAR APÓS O SPLIT)

X_train = pd.DataFrame(MinMaxScaler().fit_transform(X_train), index=X_train.index, columns=X_train.columns)
X_test =  pd.DataFrame(MinMaxScaler().fit_transform(X_test), index=X_test.index, columns=X_test.columns)


In [ ]:
# Treino dos modelos

regLinear = LinearRegression().fit(X_train,y_train)
ada = AdaBoostRegressor().fit(X_train,y_train)
gradient = GradientBoostingRegressor().fit(X_train,y_train)
randForest = RandomForestRegressor().fit(X_train,y_train)

In [ ]:
# Predição de cada modelo
yLinear = regLinear.predict(X_test)
yAda = ada.predict(X_test)
yGradient = gradient.predict(X_test)
yRandF = randForest.predict(X_test)

In [ ]:
# Avaliação e decisão do melhor modelo em relação às métricas
LinearMSE = mean_squared_error(y_test,yLinear)
AdaMSE = mean_squared_error(y_test,yAda)
GradientMSE = mean_squared_error(y_test,yGradient)
RandFMSE = mean_squared_error(y_test,yRandF)

LinearMAE = mean_absolute_error(y_test,yLinear)
AdaMAE = mean_absolute_error(y_test,yAda)
GradientMAE = mean_absolute_error(y_test,yGradient)
RandFMAE = mean_absolute_error(y_test,yRandF)

LinearR2 = r2_score(y_test,yLinear)
AdaR2 = r2_score(y_test,yAda)
GradientR2 = r2_score(y_test,yGradient)
RandFR2 = r2_score(y_test,yRandF)

print('Regressão Linear')
print('MSE:',LinearMSE)
print('RMSE:',np.sqrt(LinearMSE))
print('MAE:',LinearMAE)
print('R2:',LinearR2)
print()
print('AdaBoost')
print('MSE:',AdaMSE)
print('RMSE:',np.sqrt(AdaMSE))
print('MAE:',AdaMAE)
print('R2:',AdaR2)
print()
print('GradientBoost')
print('MSE:',GradientMSE)
print('RMSE:',np.sqrt(GradientMSE))
print('MAE:',GradientMAE)
print('R2:',GradientR2)
print()
print('Random Forest')
print('MSE:',RandFMSE)
print('RMSE:',np.sqrt(RandFMSE))
print('MAE:',RandFMAE)
print('R2:',RandFR2)
print()


Regressão Linear
MSE: 89774138.53790858
RMSE: 9474.921558403985
MAE: 7477.611781799124
R2: -0.5042228511770659

AdaBoost
MSE: 6897053.867786177
RMSE: 2626.224260756529
MAE: 1963.3996345269547
R2: 0.8844354710310893

GradientBoost
MSE: 6492631.420425926
RMSE: 2548.0642496659943
MAE: 1707.951540039463
R2: 0.8912118266358983

Random Forest
MSE: 6733020.88220577
RMSE: 2594.8065211506178
MAE: 1628.831730769231
R2: 0.8871839481457169



In [ ]:
pd.DataFrame({'real':y_test,
              'linear':yLinear,
              'ada':yAda})

,real,linear,ada
164,8238.0,3530.406311,8174.333333
103,13499.0,17032.556912,19700.063830
51,6095.0,1283.368320,7978.320000
189,11595.0,17847.753209,10028.280000
188,9995.0,11964.078207,9953.696970
170,11199.0,17392.924457,16364.425000
139,7053.0,24349.017189,8195.045455
96,7499.0,5925.387460,7978.320000
64,11245.0,4698.067427,9409.263158
101,13499.0,19394.542251,18532.920000
